In [ ]:
import os
import pandas as pd
import json
import subprocess

# Parametri principali
LABEL_COLUMN = "class"
MODEL_NAME = "llama3"     # Cambia con il modello Ollama che preferisci
N_FEW_SHOT = 5             # Numero di esempi da passare al modello
N_GEN_AT_TIME = 10             # Numero di esempi da generare

def call_ollama(prompt: str, model: str = MODEL_NAME) -> str:
    """
    Interroga Ollama in locale e restituisce la risposta testuale.
    """
    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt.encode("utf-8"),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        return result.stdout.decode("utf-8")
    except subprocess.CalledProcessError as e:
        print("❌ Errore nell'interrogazione di Ollama:", e.stderr.decode("utf-8"))
        return ""

def generate_with_llm(file, minority_examples: pd.DataFrame, n_generate: int) -> pd.DataFrame:
    """
    Chiede all'LLM di generare nuove tuple coerenti con la classe minoritaria.
    """
    # Converti i few-shot in testo leggibile
    examples_text = minority_examples.to_csv(index=False, sep=";")

    prompt = f"""
        Sei un assistente che genera nuovi esempi per bilanciare un dataset.
        Ti verranno fornite alcune tuple di riferimento (few-shot) appartenenti alla stessa classe.
        Genera {N_GEN_AT_TIME} nuove tuple realistiche e coerenti con la distribuzione dei dati.
        Restituisci SOLO un output CSV valido con lo stesso schema (stesse colonne, separatore ",m").
        
        Esempi (classe minoritaria):
        {examples_text}
        Output CSV:
        """
    response = call_ollama(prompt)
    #print(response)
    with open("./LLM/{}_{}_{}_{}.txt".format(file.replace(".csv",""),MODEL_NAME, n_generate, n_generate+N_GEN_AT_TIME), "w") as text_file:
        text_file.write("%s" % response)
    '''  
    try:
        # Leggi la risposta come CSV
        df_new = pd.read_csv(pd.compat.StringIO(response), sep=";")
        return df_new
    except Exception:
        print("⚠️ Non è stato possibile leggere la risposta come CSV, restituisco vuoto.")
        return pd.DataFrame()
    '''  

def augment_dataset(file):
    print(f"\n📂 Analizzando dataset: {file}")
    df = pd.read_csv(file)

    if LABEL_COLUMN not in df.columns:
        print(f"  ❌ Colonna '{LABEL_COLUMN}' non trovata in {file}.")
        return

    # Conta le classi
    counts = df[LABEL_COLUMN].value_counts()
    majority_class = counts.idxmax()
    minority_class = counts.idxmin()

    print(f"  🔹 Classe maggioritaria: {majority_class} ({counts[majority_class]})")
    print(f"  🔹 Classe minoritaria: {minority_class} ({counts[minority_class]})")

    # Numero di nuove tuple da generare
    diff = counts[majority_class] - counts[minority_class]
    if diff <= 0:
        print("  ✅ Dataset già bilanciato. Nessuna generazione necessaria.")
        return

    print(f"  ➕ Genererò {diff} nuove tuple per bilanciare la classe minoritaria.")

    # Preleva 5 esempi della classe minoritaria per il few-shot
    few_shot = df[df[LABEL_COLUMN] == minority_class].sample(
        min(N_FEW_SHOT, counts[minority_class]), random_state=42
    )

    # Genera nuove tuple con Ollama
    for i in range(0,diff+10,N_GEN_AT_TIME):
        print("\t Genero da {} a {}".format(i, i+N_GEN_AT_TIME))
        new_data = generate_with_llm(file, few_shot, i)

In [ ]:
from tqdm import tqdm
csv_files = [f for f in os.listdir(".") if f.endswith(".csv")]
if not csv_files:
    print("⚠️ Nessun file CSV trovato nella directory corrente.")

csv_files_already_processed = [f.split("_llama")[0] for f in os.listdir("LLM") if f.endswith(".txt")]
files_already_processed = set(csv_files_already_processed)

for file in tqdm(csv_files,total=len(csv_files)):
    print(file)
    if file.replace(".csv","") not in files_already_processed:
        augment_dataset(file)

  0%|                                                    | 0/33 [00:00<?, ?it/s]

iris0.csv
newthyroid2.csv
Migraine_onevsrest_4.csv
Migraine_onevsrest_5.csv
Migraine_onevsrest_1.csv
abalone19.csv
Migraine_onevsrest_0.csv
Migraine_onevsrest_2.csv
Migraine_onevsrest_3.csv
transfusion.csv
new-thyroid1.csv
kddcup-guess_passwd_vs_satan.csv
ecoli1.csv
cleveland-0_vs_4.csv
ecoli-0_vs_1.csv
Obesity_onevsrest_0.csv
Obesity_onevsrest_1.csv

📂 Analizzando dataset: Obesity_onevsrest_1.csv
  🔹 Classe maggioritaria: 0 (1805)
  🔹 Classe minoritaria: 1 (282)
  ➕ Genererò 1523 nuove tuple per bilanciare la classe minoritaria.
	 Genero da 0 a 10
	 Genero da 10 a 20
	 Genero da 20 a 30
	 Genero da 30 a 40
	 Genero da 40 a 50
	 Genero da 50 a 60
	 Genero da 60 a 70
	 Genero da 70 a 80
	 Genero da 80 a 90
	 Genero da 90 a 100
	 Genero da 100 a 110
	 Genero da 110 a 120
	 Genero da 120 a 130
	 Genero da 130 a 140
	 Genero da 140 a 150
	 Genero da 150 a 160
	 Genero da 160 a 170
	 Genero da 170 a 180
	 Genero da 180 a 190
	 Genero da 190 a 200
	 Genero da 200 a 210
	 Genero da 210 a 220


 52%|██████████████████████▏                    | 17/33 [19:41<18:31, 69.50s/it]

page-blocks-1-3_vs_4.csv

📂 Analizzando dataset: page-blocks-1-3_vs_4.csv
  🔹 Classe maggioritaria: 0 (443)
  🔹 Classe minoritaria: 1 (28)
  ➕ Genererò 415 nuove tuple per bilanciare la classe minoritaria.
	 Genero da 0 a 10
	 Genero da 10 a 20
	 Genero da 20 a 30
	 Genero da 30 a 40
	 Genero da 40 a 50
	 Genero da 50 a 60
	 Genero da 60 a 70
	 Genero da 70 a 80
	 Genero da 80 a 90
	 Genero da 90 a 100
	 Genero da 100 a 110
	 Genero da 110 a 120
	 Genero da 120 a 130
	 Genero da 130 a 140
	 Genero da 140 a 150
	 Genero da 150 a 160
	 Genero da 160 a 170
	 Genero da 170 a 180
	 Genero da 180 a 190
	 Genero da 190 a 200
	 Genero da 200 a 210
	 Genero da 210 a 220
	 Genero da 220 a 230
	 Genero da 230 a 240
	 Genero da 240 a 250
	 Genero da 250 a 260
	 Genero da 260 a 270
	 Genero da 270 a 280
	 Genero da 280 a 290
	 Genero da 290 a 300
	 Genero da 300 a 310
	 Genero da 310 a 320
	 Genero da 320 a 330
	 Genero da 330 a 340
	 Genero da 340 a 350
	 Genero da 350 a 360
	 Genero da 360 a 370


 55%|███████████████████████▍                   | 18/33 [23:01<19:54, 79.64s/it]

Obesity_onevsrest_3.csv

📂 Analizzando dataset: Obesity_onevsrest_3.csv
  🔹 Classe maggioritaria: 0 (1797)
  🔹 Classe minoritaria: 1 (290)
  ➕ Genererò 1507 nuove tuple per bilanciare la classe minoritaria.
	 Genero da 0 a 10
	 Genero da 10 a 20
	 Genero da 20 a 30
	 Genero da 30 a 40
	 Genero da 40 a 50
	 Genero da 50 a 60
	 Genero da 60 a 70


In [49]:
csv_files_already_processed = [f.split("_llama")[0] for f in os.listdir("LLM") if f.endswith(".txt")]
files_already_processed = set(csv_files_already_processed)
for file in csv_files:
    if file.replace(".csv","") not in files_already_processed:
        print(file)

Obesity_onevsrest_1.csv
page-blocks-1-3_vs_4.csv
Obesity_onevsrest_3.csv
Obesity_onevsrest_2.csv
Obesity_onevsrest_6.csv
pima.csv
Obesity_onevsrest_5.csv
abalone9-18.csv
Obesity_onevsrest_4.csv
yeast1.csv
dermatology-6.csv
yeast3.csv
vehicle2.csv
vehicle3.csv
vehicle1.csv
vehicle0.csv
vowel0.csv


In [46]:
def call_ollama(prompt: str, model: str = MODEL_NAME) -> str:
    """
    Interroga Ollama in locale e restituisce la risposta testuale.
    """
    try:
        result = subprocess.run(
            ["ollama", "run", model],
            input=prompt.encode("utf-8"),
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            check=True
        )
        return result.stdout.decode("utf-8")
    except subprocess.CalledProcessError as e:
        print("❌ Errore nell'interrogazione di Ollama:", e.stderr.decode("utf-8"))
        return ""

In [47]:
response = call_ollama("Ciao sono stefano")
response

'Ciao Stefano! Sono felice di conoscerti. Come posso aiutarti oggi?\n\n'